In [13]:
import scanpy as sc
import anndata as an
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder 
import torch
from sys import getsizeof
import json 

import os, sys
sys.path.append('../src')
from models.lib.data import *
from models.lib.lightning_train import DataModule, generate_trainer

datafiles=['../data/mouse/MouseAdultInhibitoryNeurons.h5ad']
labelfiles=['../data/mouse/Adult Inhibitory Neurons in Mouse_labels.tsv']

le = LabelEncoder()
pd.read_csv('../data/mouse/subset_labels.csv')

Unnamed: 0   cell                                        index  \
0             0      0  AAAAAAAAAAAAAA_p25-27_Amygdala_SAMN08730984   
1             1      1  GAAACCCAATCTCG_p25-27_Amygdala_SAMN08730984   
2             2     62  TAATGATGAAACAG_p25-27_Amygdala_SAMN08730984   
3             3     79     CATTGTACAGGAGC_p29_SScortex_SAMN08730874   
4             4    103     ACGCAATGGGTTAC_p29_SScortex_SAMN08730874   
..          ...    ...                                          ...   
495         495  24793     TAGTTAGATCGACA_p29_Amygdala_SAMN08730938   
496         496  24864     TCGCAAGAGTTTGG_p29_Amygdala_SAMN08730938   
497         497  24963         GAAACCCAATCTCG_p27_Ctx2_SAMN08730922   
498         498  25078         ATCCTAACTGTCGA_p27_Ctx2_SAMN08730922   
499         499  25160     AACCTTACAGCTCA_p28_SScortex_SAMN08730873   

                 class  numeric_class  
0    S-phase_MCM4/H43C              1  
1    S-phase_MCM4/H43C              1  
2    S-phase_MCM4/H43C              1  
3    S-phase_MCM4/H43C              1  
4    S-phase_MCM4/H43C              1  
..                 ...            ...  
495  S-phase_MCM4/H43C              1  
496  S-phase_MCM4/H43C              1  
497  S-phase_MCM4/H43C              1  
498  S-phase_MCM4/H43C              1  
499  S-phase_MCM4/H43C              1  

[500 rows x 5 columns]

In [2]:
mouse_atlas = sc.read_h5ad('../data/mouse/MouseAdultInhibitoryNeurons.h5ad')

In [9]:
atlas_labels.reset_index()

index              class  \
0             AAAAAAAAAAAAAA_p25-27_Amygdala_SAMN08730984  S-phase_MCM4/H43C   
1             GAAACCCAATCTCG_p25-27_Amygdala_SAMN08730984  S-phase_MCM4/H43C   
2             GACCAAACTGCCTC_p25-27_Amygdala_SAMN08730984       Ctx_LHX6/SST   
3             GCGTAATGGACGGA_p25-27_Amygdala_SAMN08730984      Str_LHX8/CHAT   
4             CTTGAGGACAGAAA_p25-27_Amygdala_SAMN08730984      Str_LHX8/CHAT   
...                                                   ...                ...   
141064  ACTACGGACAGAAA_e12.0_ForebrainVentral_SRR11947...  S-phase_MCM4/H43C   
141065  GGGACCTGGCGAAG_e12.0_ForebrainVentral_SRR11947...         Transition   
141066  CAACAGACTGGTTG_e12.0_ForebrainVentral_SRR11947...         Transition   
141067  AAAGATCTCTTCCG_e12.0_ForebrainVentral_SRR11947...  S-phase_MCM4/H43C   
141068  ATGTTAGACACAAC_e12.0_ForebrainVentral_SRR11947...  S-phase_MCM4/H43C   

        numeric_class  
0                  36  
1                  36  
2                   9  
3                  40  
4                  40  
...               ...  
141064             36  
141065             41  
141066             41  
141067             36  
141068             36  

[141069 rows x 3 columns]

In [6]:
atlas_labels = pd.DataFrame(mouse_atlas.obs['class']).reset_index()
le = LabelEncoder()
atlas_labels['numeric_class'] = le.fit_transform(atlas_labels['class'])

subset_labels = atlas_labels[atlas_labels['class'].isin(['S-phase_MCM4/H43C', 'MGE_LHX6/MAF'])]
subset_labels = subset_labels.iloc[0: 500, :]
subset_labels.index.name = 'cell'

subset_labels.to_csv('../data/mouse/subset_labels.csv')
pd.read_csv('../data/mouse/subset_labels.csv')

NameError: name 'mouse_atlas' is not defined

In [5]:
total_class_weights(['../data/mouse/subset_labels.csv'], 'numeric_class')

tensor([7.5758, 0.5353])

In [5]:
# atlas_labels.to_csv('../data/mouse/MouseAdultInhibitoryNeurons_labels.csv', index=False)

# pd.read_csv('../data/mouse/MouseAdultInhibitoryNeurons_labels.csv')

In [12]:
mouse_atlas = sc.read_h5ad('../data/mouse/MouseAdultInhibitoryNeurons.h5ad')
mo_data = an.read_h5ad('../data/mouse/Mo_PV_paper_TDTomato_mouseonly.h5ad')

g1 = mo_data.var.index.values 
g2 = mouse_atlas.var.index.values

refgenes = sorted(list(set(g1).intersection(g2)))

Variable names are not unique. To make them unique, call `.var_names_make_unique`.


## 

In [13]:
g1, g2, refgenes

(array(['Xkr4', 'Gm1992', 'Gm19938', ..., 'AC234645.1', 'AC149090.1',
        'TDTomato'], dtype=object),
 array(['0610005C13Rik', '0610006L08Rik', '0610009B22Rik', ..., 'n-R5s95',
        'n-R5s96', 'n-TStga1'], dtype=object),
 ['0610005C13Rik',
  '0610006L08Rik',
  '0610009B22Rik',
  '0610009E02Rik',
  '0610009L18Rik',
  '0610010F05Rik',
  '0610010K14Rik',
  '0610012D04Rik',
  '0610012G03Rik',
  '0610025J13Rik',
  '0610030E20Rik',
  '0610031O16Rik',
  '0610033M10Rik',
  '0610038B21Rik',
  '0610039K10Rik',
  '0610040B10Rik',
  '0610040F04Rik',
  '0610040J01Rik',
  '0610043K17Rik',
  '0710001A04Rik',
  '1010001B22Rik',
  '1110002E22Rik',
  '1110002J07Rik',
  '1110002L01Rik',
  '1110002O04Rik',
  '1110004F10Rik',
  '1110006O24Rik',
  '1110008P14Rik',
  '1110012L19Rik',
  '1110013H19Rik',
  '1110015O18Rik',
  '1110017D15Rik',
  '1110018N20Rik',
  '1110019D14Rik',
  '1110020A21Rik',
  '1110028F11Rik',
  '1110032A03Rik',
  '1110032F04Rik',
  '1110035H17Rik',
  '1110038B12Rik',
  '1110038F1

# Mouse model training with generate_trainer test

In [14]:
datafiles

['../data/mouse/MouseAdultInhibitoryNeurons.h5ad']

In [15]:
trainer, model, module = generate_trainer(
    datafiles=['../data/mouse/MouseAdultInhibitoryNeurons.h5ad'],
    labelfiles=['../data/mouse/subset_labels.csv'],
    class_label='numeric_class',
    drop_last=True,
    shuffle=True,
    batch_size=4,
    num_workers=0,
    refgenes=refgenes,
    weighted_metrics=True,
    optim_params={
        'optimizer': torch.optim.Adam,
        'lr': 0.02,
        'weight_decay': 3e-4,
    },
    max_epochs=1000,
    collocate=False,
)

# Mouse Model training code using lib.lightning_data

In [22]:
labelfile = '../data/mouse/MouseAdultInhibitoryNeurons_labels.csv'
label_df = pd.read_csv(labelfile)


label_df

class  numeric_class
0       S-phase_MCM4/H43C             36
1       S-phase_MCM4/H43C             36
2            Ctx_LHX6/SST              9
3           Str_LHX8/CHAT             40
4           Str_LHX8/CHAT             40
...                   ...            ...
141064  S-phase_MCM4/H43C             36
141065         Transition             41
141066         Transition             41
141067  S-phase_MCM4/H43C             36
141068  S-phase_MCM4/H43C             36

[141069 rows x 2 columns]

In [ ]:
test_prop=0.2
# trainsplit, valsplit = train_test_split(current_labels, stratify=current_labels, test_size=test_prop)
# trainsplit, testsplit = train_test_split(trainsplit, stratify=trainsplit, test_size=test_prop)


# trainsplit.index

In [23]:
module = DataModule(
    datafiles=['../data/mouse/MouseAdultInhibitoryNeurons.h5ad'],
    labelfiles=['../data/mouse/MouseAdultInhibitoryNeurons_labels.csv'],
    class_label='numeric_class',
    collocate=False,
    batch_size=4,
    num_workers=0,
    refgenes=refgenes,
    currgenes=g2,
)

module

In [ ]:
module.setup()

Creating train/val/test DataLoaders...


/Users/julian/Documents/Projects/organoid-classification/notebooks/../src/models/lib/data.py:601: UserWarning: collocate =False, so multiple files will return multiple DataLoaders and cannot be trained sequentially with PyTorch-Lightning
  warnings.warn(f"{collocate =}, so multiple files will return multiple DataLoaders and cannot be trained sequentially with PyTorch-Lightning")


In [ ]:
train = module.train_dataloader()

print(next(iter(train)))

In [ ]:
train = DataLoader(atlas_train, batch_size=4, num_workers=0, drop_last=True, shuffle=True)
val = DataLoader(atlas_val, batch_size=4, num_workers=0, drop_last=True, shuffle=False)

In [ ]:
# from models.lib.neural import GeneClassifier
# from pytorch_lightning import Trainer 
# from pytorch_lightning.loggers import WandbLogger

# model = GeneClassifier(
#     input_dim=34430, 
#     output_dim=50,
#     optim_params={
#         'optimizer': torch.optim.Adam,
#         'lr': 3e-4,
#     }
# )

# wandb_logger = WandbLogger(project='Mouse Classifier', name='Tabnet with Metrics')
# trainer = Trainer(logger=wandb_logger)

# trainer.fit(model, train, val)

In [ ]:
def gene_intersection(
    files
):
    import dask.dataframe as dd 
    
    cols = []
    for file in files:
        temp = pd.read_csv(fpath, nrows=1, header=1).columns 
        temp = [x.split('|')[0].upper().strip() for x in temp]
        cols.append(set(temp))
    
    unique = list(set.intersection(*cols))
    unique = sorted(unique)
    
    return unique 

In [ ]:
generate_single_dataset(
    datafile='../data/mouse/MouseAdultInhibitoryNeurons.h5ad',
    labelfile='../data/mouse/Adult Inhibitory Neurons in Mouse_labels.tsv',
    class_label='numeric_class',
    sep='\t'
)